# Comparing groundstate wigner functions 

In this notebook we look at how the RWA breaks down in the JC Hamiltonian by comparing their Wigner functions. By plotting the Wigner function of the ground state, we can see that the RWA no longer gives a correct result if we keep increasing the coupling. For these models to give a correct result, a larger Hilbert space is needed because of the extremely high coupling.

What we see in the Wigner plots is that for the full Hamiltonian, the functions evolve into a “cat”-like state at high coupling. It splits into two dots with some interference fringes, a characteristic of superposed coherent states.

The Wigner function of the RWA shows a ground state that does not evolve smoothly. Instead, as $g$ increases, we can see multiple peaks appear. This happens because higher photon-number states are added step by step in the JC ground state. Unlike the full Hamiltonian, the RWA ground state does not show interference fringes or a coherent superposition. This shows a limit of the RWA at strong coupling and why it fails as the interaction gets stronger.

In [ ]:
#import libraries
from qutip import *
from qutip.wigner import wigner
from qutip.visualization import plot_wigner
from matplotlib import transforms
from IPython.display import display, HTML
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import matplotlib.transforms as transforms

#Hamiltonians
def HamiltonianJaynesCummings(g : float, wr : float, wt : float, N : int) -> Qobj:
    a = destroy(N)
    return tensor(a.dag()*a, qeye(2))*wr + 0.5*wr - 0.5*wt*tensor(qeye(N), sigmaz()) + g*(tensor(a.dag(),sigmap()) + tensor(a,sigmam()))

def HamiltonianRabi(g : float, wr : float, wt : float, N : int) -> Qobj:
    a = destroy(N)
    return tensor(a.dag()*a, qeye(2))*wr + 0.5*wr - 0.5*wt*tensor(qeye(N), sigmaz()) + g*tensor(a.dag() + a, qeye(2))*tensor(qeye(N), sigmap() + sigmam())

#wigner plot function 
def plot_wigner_psi_phi(psi, fig, alpha_max=7.5):
    fig.clf()  # clear figure
    
    widths = [6,3]
    heights = [6,3]
    spec = fig.add_gridspec(ncols=2, nrows=2, width_ratios=widths, height_ratios=heights)
    
    x = np.linspace(-alpha_max, alpha_max, 200)
    wig = wigner(psi, x, x)
    
    psi_x = np.sum(wig, axis=0)
    psi_p = np.sum(wig, axis=1)
    
    # making a wigner subplot 
    ax_wig = fig.add_subplot(spec[0,0])
    plot_wigner(psi, x, x, fig=fig, ax=ax_wig)
    ax_wig.set_title('Wigner function')
    
    # p distribution 
    ax_p = fig.add_subplot(spec[0,1])
    base = ax_p.transData
    rot = transforms.Affine2D().rotate_deg(90)
    ax_p.plot(x, -psi_p, transform=rot + base)
    ax_p.set_xticks([])
    ax_p.set_ylim(-alpha_max, alpha_max)
    ax_p.set_title('p distribution')
    
    # x distribution
    ax_x = fig.add_subplot(spec[1,0])
    ax_x.plot(x, psi_x)
    ax_x.set_yticks([])
    ax_x.set_xlim(-alpha_max, alpha_max)
    ax_x.set_title('x distribution')
    
    fig.tight_layout()
    return fig

#wigner plot function with marginals side by side
def plot_wigner_psi_phi_axes(psi, fig, grid_position, alpha_max=7.5, title=""):
    widths = [6,3]
    heights = [6,3]
    spec = fig.add_gridspec(ncols=4, nrows=2, width_ratios=[6,3,6,3], height_ratios=heights)
    
    x = np.linspace(-alpha_max, alpha_max, 200)
    wig = wigner(psi, x, x)
    psi_x = np.sum(wig, axis=0)
    psi_p = np.sum(wig, axis=1)
    
    # Wigner subplot
    ax_wig = fig.add_subplot(spec[0, grid_position])
    plot_wigner(psi, x, x, fig=fig, ax=ax_wig)
    ax_wig.set_title(title)
    
    # p distribution
    ax_p = fig.add_subplot(spec[0, grid_position+1])
    base = ax_p.transData
    rot = transforms.Affine2D().rotate_deg(90)
    ax_p.plot(x, -psi_p, transform=rot + base)
    ax_p.set_xticks([])
    ax_p.set_ylim(-alpha_max, alpha_max)
    ax_p.set_title('p distribution')
    
    # x distribution
    ax_x = fig.add_subplot(spec[1, grid_position])
    ax_x.plot(x, psi_x)
    ax_x.set_yticks([])
    ax_x.set_xlim(-alpha_max, alpha_max)
    ax_x.set_title('x distribution')
    
    return fig

#parameters 
N = 50
wr = 1.0
wt = 1.0
g_values = np.linspace(0.0, 4.0, 30)

#animation
fig = plt.figure(figsize=(12,6))

def animate(i):
    fig.clf()
    g = g_values[i]
    H_JC = HamiltonianJaynesCummings(g, wr, wt, N)
    psi_jc = H_JC.groundstate()[1]
    rho_jc = ket2dm(psi_jc).ptrace(0)
    H_Rabi = HamiltonianRabi(g, wr, wt, N)
    psi_rabi = H_Rabi.groundstate()[1]
    rho_rabi = ket2dm(psi_rabi).ptrace(0)
    plot_wigner_psi_phi_axes(rho_jc, fig, grid_position=0, title=f"RWA H g={g:.2f}")
    plot_wigner_psi_phi_axes(rho_rabi, fig, grid_position=2, title=f"Full H g={g:.2f}")
    fig.tight_layout()
    fig.subplots_adjust(hspace=0.3, wspace=0.4)
    return fig,

ani = FuncAnimation(fig, animate, frames=len(g_values), interval=500, blit=False)
html_output = ani.to_jshtml()
plt.close(fig)
HTML(html_output)

/usr/local/python/3.12.1/lib/python3.12/site-packages/qutip/core/qobj.py:1810: UserWarning: Ground state may be degenerate.
  warnings.warn("Ground state may be degenerate.", UserWarning)
